In [1]:
# importing libraries
import os
import openai
import pinecone
# from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

c:\Users\hetka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import pandas as pd

def load_dataset(file_path):
    dataset = pd.read_csv(file_path)
    return dataset

def convert_csv_to_document(dataset):
    document = ''
    for index, row in dataset.iterrows():
        row_values = row.values.tolist()
        row_text = ' '.join(str(value) for value in row_values)
        document += row_text + '\n'
    return document

file_path = './dataset.csv'  # Update with the actual file path
dataset = load_dataset(file_path)

document = convert_csv_to_document(dataset)
document


"1 Laptop Powerful laptop with a sleek design, ideal for both work and entertainment Electronics 999.99 4.5 1 John Doe I absolutely love my new laptop! It's super fast and the battery life is amazing.\n1 Laptop Powerful laptop with a sleek design, ideal for both work and entertainment Electronics 999.99 4.5 2 Alice Smith Great value for money. The laptop performs well and is perfect for both work and entertainment.\n1 Laptop Powerful laptop with a sleek design, ideal for both work and entertainment Electronics 999.99 4.5 3 Emily Johnson The design is sleek and modern. However, the trackpad could be more responsive.\n2 Smartphone High-performance smartphone with a stunning display and long battery life Electronics 799.99 4.3 4 Michael Brown This smartphone has an excellent camera and the battery life is impressive.\n2 Smartphone High-performance smartphone with a stunning display and long battery life Electronics 799.99 4.3 5 Sarah Wilson I'm very satisfied with my purchase. The phone's

In [3]:
def split_docs(documents, chunk_size=200, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_text(documents)
  return docs

In [4]:
docs = split_docs(document)

In [5]:
docs

["1 Laptop Powerful laptop with a sleek design, ideal for both work and entertainment Electronics 999.99 4.5 1 John Doe I absolutely love my new laptop! It's super fast and the battery life is amazing.",
 '1 Laptop Powerful laptop with a sleek design, ideal for both work and entertainment Electronics 999.99 4.5 2 Alice Smith Great value for money. The laptop performs well and is perfect for both work',
 'for both work and entertainment.',
 '1 Laptop Powerful laptop with a sleek design, ideal for both work and entertainment Electronics 999.99 4.5 3 Emily Johnson The design is sleek and modern. However, the trackpad could be more',
 'could be more responsive.',
 '2 Smartphone High-performance smartphone with a stunning display and long battery life Electronics 799.99 4.3 4 Michael Brown This smartphone has an excellent camera and the battery life is',
 'the battery life is impressive.',
 "2 Smartphone High-performance smartphone with a stunning display and long battery life Electronics 7

In [6]:
embeddings = OpenAIEmbeddings()

c:\Users\hetka\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
from langchain.vectorstores import Chroma
db = Chroma.from_texts(docs, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})

In [8]:
import google.generativeai as genai

GOOGLE_API_KEY='' #create your own api key
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

In [9]:
import os
os.environ["OPENAI_API_KEY"] = ""#create your own api key

In [10]:
from langchain.chat_models import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI()

c:\Users\hetka\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

In [12]:
from langchain.prompts import PromptTemplate # for custom prompt specification
from langchain.text_splitter import RecursiveCharacterTextSplitter # splitter for chunks
from langchain.chains import RetrievalQA # qa and retriever chain
from langchain.memory import ConversationBufferMemory # for model's memory on past conversations


qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)
        
        
query = "what is a good product in laptop?"
result = qa({"query": query})

c:\Users\hetka\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [13]:
result

{'query': 'what is a good product in laptop?',
 'result': 'Based on the provided context, the laptop mentioned seems to be a good product. It is described as powerful, sleek, ideal for work and entertainment, fast, and with great battery life. Customers John Doe and Alice Smith have both expressed positive feedback about this laptop.',
 'source_documents': [Document(page_content="1 Laptop Powerful laptop with a sleek design, ideal for both work and entertainment Electronics 999.99 4.5 1 John Doe I absolutely love my new laptop! It's super fast and the battery life is amazing."),
  Document(page_content='1 Laptop Powerful laptop with a sleek design, ideal for both work and entertainment Electronics 999.99 4.5 2 Alice Smith Great value for money. The laptop performs well and is perfect for both work')]}

In [14]:
from gtts import gTTS

In [15]:
def text_to_speech(text, output_file="output.mp3"):
    tts = gTTS(text=text, lang="en")
    tts.save(output_file)
    return output_file

In [16]:
text = result['result']
output_file = text_to_speech(text)
os.system(f"start {output_file}") 

0

In [17]:
import speech_recognition as sr

def speech_to_text(duration=10):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening for speech...")
        audio = recognizer.listen(source, timeout=duration)  # Listen for audio input for a specified duration
    
    try:
        text = recognizer.recognize_google(audio)  # Recognize the speech using Google Speech Recognition
        return text
    except sr.UnknownValueError:
        return "Speech recognition could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"

# Example usage:
duration = 10  # Duration to listen for speech input in seconds
text = speech_to_text(duration)
print("Transcribed Text:", text)

Listening for speech...


KeyboardInterrupt: 